In [321]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select

from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def find_text_by_class(obj, html_tag, html_class):
    return obj.find(html_tag, {'class': html_class}).text.strip()

def find_all_by_class(obj, html_tag, html_class):
    return obj.find_all(html_tag, {'class': html_class})

In [ ]:
def get_data_disciplina(disciplina):
    disciplina_dict = dict()
    
    text_disciplina = disciplina.find('span', {'class': 'tituloDisciplina'}).text.strip()
    disciplina_dict['cod'] = text_disciplina.split(" - ")[0]
    disciplina_dict['nome'] = text_disciplina.split(" - ")[1]
    
    return disciplina_dict

def get_data_turma(turma_obj):
    turma_dict = dict()
    
    turma_dict['turma'] = find_text_by_class(turma_obj, 'td', 'turma')
    turma_dict['periodo'] = find_text_by_class(turma_obj, 'td', 'anoPeriodo')
    turma_dict['professor'] = find_text_by_class(turma_obj, 'td', 'nome')
    turma_dict['horario'] = find_all_by_class(turma_obj, 'td', None)[0].text.strip().split('\n')[0]
    turma_dict['vagas_ocupadas'] = find_all_by_class(turma_obj, 'td', None)[3].text.strip()
    turma_dict['total_vagas'] = find_all_by_class(turma_obj, 'td', None)[2].text.strip()
    turma_dict['local'] = find_all_by_class(turma_obj, 'td', None)[4].text.strip()
    
    return turma_dict

In [5]:
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)

<ipython-input-5-361060fc6bf4>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)


In [322]:
DRIVER_PATH = './selenium/chromedriver'
ser = Service(DRIVER_PATH)
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ser, options=op)

In [324]:
driver.get("https://sigaa.unb.br/sigaa/public/turmas/listar.jsf")

In [326]:
main_page_sigaa = driver.page_source

In [328]:
main_soup = BeautifulSoup(main_page_sigaa, 'html.parser')

In [334]:
departamentos = main_soup.find('select', {'id': 'formTurma:inputDepto'})

In [340]:
departamentos = departamentos.find_all('option', {'selected': None})

In [350]:
departamentos_dict = {
    'cod': [],
    'nome': []
}

In [351]:
for departamento in departamentos:
    departamentos_dict['cod'].append(departamento.get('value'))
    departamentos_dict['nome'].append(departamento.text)

In [359]:
depto_sources = dict()

In [363]:
len(departamentos_dict['cod'])

204

In [364]:
for departamento in departamentos_dict['cod']:
    driver.get("https://sigaa.unb.br/sigaa/public/turmas/listar.jsf")
    
    select = Select(driver.find_element('id', 'formTurma:inputDepto'))
    select.select_by_value(departamento)

    elem = driver.find_element('xpath', "//input[@id='formTurma:inputAno']")
    elem.clear()
    elem.send_keys('2022')
    
    select = Select(driver.find_element('id', 'formTurma:inputPeriodo'))
    select.select_by_value('2')
    
    elem = driver.find_element('xpath', "//input[@name='formTurma:j_id_jsp_1370969402_11']")
    elem.click()
    
    depto_sources[departamento] = driver.page_source
    
    print(len(depto_sources.keys()))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204


In [386]:
disciplinas_dict_geral = []
turmas_dict_geral = []

for k in depto_sources.keys():
    soup = BeautifulSoup(depto_sources[k], 'html.parser')
    
    disciplinas = soup.find_all('tr', {"class": "agrupador"})
    
    turmas_par = soup.find_all('tr', {"class": "linhaPar"})
    turmas_impar = soup.find_all('tr', {"class": "linhaImpar"})
    turmas = sorted(turmas_par + turmas_impar, key=lambda x: x.sourceline)
    
    disciplinas_turmas = []

    for i in range(0, len(disciplinas)):
        disciplinas_turmas.append([disciplinas[i], []])

        pos_disciplina = disciplinas[i].sourceline

        if i != (len(disciplinas)-1):
            pos_prox_disciplina = disciplinas[i+1].sourceline
        else:
            pos_prox_disciplina = None

        if pos_prox_disciplina != None:
            for turma in turmas:
                if (turma.sourceline > pos_disciplina) and (turma.sourceline < pos_prox_disciplina):
                    disciplinas_turmas[i][1].append(turma)
        else:
            for turma in turmas:
                if (turma.sourceline > pos_disciplina):
                    disciplinas_turmas[i][1].append(turma)

    disciplinas_dict = {
        'cod': [],
        'nome': []
    }

    turmas_dict = {
        'turma': [],
        'periodo': [],
        'professor': [],
        'horario': [],
        'vagas_ocupadas': [],
        'total_vagas': [],
        'local': [],
        'cod_disciplina': []
    }

    for disciplina_turmas in disciplinas_turmas:
        data_disciplina = get_data_disciplina(disciplina_turmas[0])
        data_turmas = [get_data_turma(turma) for turma in disciplina_turmas[1]]

        disciplinas_dict['cod'].append(data_disciplina['cod'])
        disciplinas_dict['nome'].append(data_disciplina['nome'])

        for turma_dict in data_turmas:
            turmas_dict['turma'].append(turma_dict['turma'])
            turmas_dict['periodo'].append(turma_dict['periodo'])
            turmas_dict['professor'].append(turma_dict['professor'])
            turmas_dict['horario'].append(turma_dict['horario'])
            turmas_dict['vagas_ocupadas'].append(turma_dict['vagas_ocupadas'])
            turmas_dict['total_vagas'].append(turma_dict['total_vagas'])
            turmas_dict['local'].append(turma_dict['local'])
            turmas_dict['cod_disciplina'].append(data_disciplina['cod'])
    
    disciplinas_df = pd.DataFrame(disciplinas_dict)
    disciplinas_df['cod_depto'] = k
    turmas_df = pd.DataFrame(turmas_dict)
    turmas_df['cod_depto'] = k
    
    disciplinas_dict_geral.append(disciplinas_df)
    turmas_dict_geral.append(turmas_df)

In [387]:
disciplinas_geral = pd.concat(disciplinas_dict_geral, ignore_index=True)
turmas_geral = pd.concat(turmas_dict_geral, ignore_index=True)